<a href="https://colab.research.google.com/github/rfaoktvian/AnalisisSentimenDebatCapres2024/blob/main/Analisis_sentimen_debat_capres2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.7 MB/s eta 0:00:00


In [13]:
import os, csv, requests, re, nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from googleapiclient.discovery import build
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [14]:
df = pd.read_csv('youtube_comments_debat_capres2024.csv')
df.head(100)

,comment_text,like_count,reply_count,published_at
0,Menjatuhkan kinerja seseorg dipublik sama dgñ ...,1,0,2026-01-21T17:42:14Z
1,Aku terbangun jam 2 mlm tiba2 ingat debat ini,1,0,2026-01-15T19:17:31Z
2,this year everything is proven,2,0,2026-01-14T05:12:17Z
3,"gua balik lagi kesini, gara gara konoha mulai ...",12,2,2025-12-20T16:19:49Z
4,Hanya Pak Anis yg bahas isu lingkungan di deba...,4,0,2025-12-14T12:47:20Z
...,...,...,...,...
95,"Kayak nyari"" kesalahan",0,0,2024-02-17T14:35:40Z
96,Ini Anis parah banget terlalu menyerang pribad...,0,0,2024-02-17T14:30:53Z
97,Flash back liat debat presiden ini\n01 & 03 pr...,0,0,2024-02-17T14:28:24Z
98,"Ini Anis ngapain nanyak pribadi ya ,kampret bnget",0,0,2024-02-17T12:30:33Z


In [15]:
df.shape

(10930, 4)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10930 entries, 0 to 10929
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment_text  10930 non-null  object
 1   like_count    10930 non-null  int64 
 2   reply_count   10930 non-null  int64 
 3   published_at  10930 non-null  object
dtypes: int64(2), object(2)
memory usage: 341.7+ KB


In [17]:
clean_df = df.drop_duplicates()

In [18]:
clean_df.duplicated().sum()

np.int64(0)

In [19]:
# Text Preprocessing

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r"http\S+", '', text)
  text = re.sub(r'[0-9]+', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  text = text.replace('\n', ' ')
  text = text.strip(' ')
  return text

slangwords = {
    "abis": "habis",
    "bgt": "banget",
    "bangettt": "banget",
    "bgtt": "banget",
    "gk": "tidak",
    "ga": "tidak",
    "gak": "tidak",
    "nggak": "tidak",
    "enggak": "tidak",
    "tdk": "tidak",
    "dr": "dari",
    "dri": "dari",
    "aja": "saja",
    "udh": "sudah",
    "udah": "sudah",
    "blm": "belum",
    "belom": "belum",
    "pdhl": "padahal",
    "krn": "karena",
    "sm": "sama",
    "jg": "juga",
    "tp": "tapi",
    "trs": "terus",
    "org": "orang",
    "kmrn": "kemarin",
    "skrg": "sekarang",
    "omon": "bicara",
    "sory": "sorry",
    "sori": "sorry",
    "anis": "anies",
    "kren": "keren",
    "sy": "saya",
    "sya": "saya",
    "nnti": "nanti",
    "yg": "yang",
    "dgn": "dengan",
    "sdh": "sudah",
    "pd": "pada",
    "bs": "bisa",
    "klo": "kalau",
    "kalo": "kalau",
    "kl": "kalau",
    "msh": "masih",
    "lg": "lagi",
    "jd": "jadi",
    "nih": "ini",
    "tuh": "itu",
    "kyk": "seperti",
    "kayak": "seperti",
    "gw": "saya",
    "gua": "saya",
    "lu": "kamu",
    "lo": "kamu",
    "tau": "tahu",
    "cm": "cuma",
    "dlm": "dalam",
    "gemoy": "gemas",
    "ny": "nya"
}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word in slangwords:
            fixed_words.append(slangwords[word])
        else:
            fixed_words.append(word)

    return " ".join(fixed_words)

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(tokens):
    stop_id = set(stopwords.words('indonesian'))
    stop_en = set(stopwords.words('english'))

    negation_words = {"tidak", "bukan", "jangan", "nggak", "enggak"}

    stopwords_all = (stop_id | stop_en) - negation_words

    custom_stopwords = {
        "iya","yaa","sih","loh","kah","woi","woii","woy",
        "nya","ku","di","ke","dari"
    }

    stopwords_all.update(custom_stopwords)

    return [word for word in tokens if word not in stopwords_all]

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmingText(text):
    return stemmer.stem(text)


In [20]:

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

def preprocess_wrapper(text):
    if not isinstance(text, str):
        return ""

    text = cleaningText(text)
    text = casefoldingText(text)
    text = fix_slangwords(text)
    tokens = tokenizingText(text)
    tokens = filteringText(tokens)
    sentence = toSentence(tokens)
    # Stemming opsional: aktifkan jika butuh akurasi lebih tapi siap menunggu lama
    # sentence = stemmingText(sentence)
    return sentence

df['final_text'] = df['comment_text'].apply(preprocess_wrapper)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
print(df[['comment_text', 'final_text']].head())

                                        comment_text  \
0  Menjatuhkan kinerja seseorg dipublik sama dgñ ...   
1      Aku terbangun jam 2 mlm tiba2 ingat debat ini   
2                     this year everything is proven   
3  gua balik lagi kesini, gara gara konoha mulai ...   
4  Hanya Pak Anis yg bahas isu lingkungan di deba...   

                                          final_text  
0  menjatuhkan kinerja seseorg dipublik dgñ menja...  
1                            terbangun jam mlm debat  
2                             year everything proven  
3          kesini gara gara konoha hancur berantakan  
4  anies bahas isu lingkungan debat capres haya t...  
